In [20]:
#ENTREGA 2
# Preparacao dos dados para analise

import pandas as pd

# 1. Carregar os dados
# Lendo os 4 arquivos csv que a gente vai usar.
df_order = pd.read_csv('/content/Order_semicolon.csv', sep=';')
df_campaign_queue = pd.read_csv('/content/CampaignQueue_semicolon.csv', sep=';')
df_campaign = pd.read_csv('/content/Campaign_semicolon.csv', sep=';')
print("...arquivos carregados.")

Carregando arquivos...
...arquivos carregados.


In [21]:
# Limpar e arrumar os dados

# convertendo colunas para tipo de data
df_order[['createdAt', 'updatedAt', 'scheduledAt']] = df_order[['createdAt', 'updatedAt', 'scheduledAt']].apply(pd.to_datetime, dayfirst=True, errors='coerce')
df_campaign_queue[['scheduledAt', 'sendAt', 'createdAt', 'updatedAt']] = df_campaign_queue[['scheduledAt', 'sendAt', 'createdAt', 'updatedAt']].apply(pd.to_datetime, dayfirst=True, errors='coerce')

# id do cliente precisa ser do mesmo tipo pra poder juntar as tabelas
df_order['customer'] = df_order['customer'].astype(str)
df_campaign_queue['customerId'] = df_campaign_queue['customerId'].astype(str)

print("...dados limpos.")


Limpando os dados...
...dados limpos.


In [22]:
# criando colunas novas pra ajudar na analise

# derivar hora e dia da semana para analisar horario de pico e vendas na semana
df_order['hora_do_dia'] = df_order['createdAt'].dt.hour
df_order['dia_da_semana'] = df_order['createdAt'].dt.day_name()

# derivando o atraso das campanhas - horaraio agendado & horario que foi enviado a mensagem
df_campaign_queue['atraso_envio'] = df_campaign_queue['sendAt'] - df_campaign_queue['scheduledAt']

print("...novas colunas criadas.")


Derivando novas colunas...
...novas colunas criadas.


In [25]:
# integrar as tabelas
# juntar tudo numa tabela só pra conseguir cruzar as informações.

# Adicionamos sufixos aqui para evitar problemas.
df_full_campaign_info = pd.merge(df_campaign_queue, df_campaign, left_on='campaignId', right_on='id', how='left', suffixes=('_queue', '_campaign'))

# Juntando os pedidos com as infos das campanhas.
# juntando apenas clientes que tem o id na tabela de camapanhas - para saber o desempenho da campanha
df_integrated = pd.merge(df_order, df_full_campaign_info, left_on='customer', right_on='customerId', how='inner', suffixes=('_order', '_campaign'))

print(f"...tabelas integradas. A tabela final tem {df_integrated.shape[0]} linhas.")


Integrando tabelas...
...tabelas integradas. A tabela final tem 10039 linhas.


In [28]:
# Preparando os resultados para cada uma das análises.

#----- 1: Horários de Pico (pedidos por hora) ---
analise_pico_horario = df_order.groupby('hora_do_dia').size().reset_index(name='qtd_pedidos')

# ---- 2: Vendas por Dia da Semana ---
analise_dia_semana = df_order['dia_da_semana'].value_counts().reset_index()
analise_dia_semana.columns = ['Dia da Semana', 'Qtd de Pedidos']

# ---- 3: Tipo de Pedido por Canal de Venda ---
analise_tipo_canal = df_order.groupby(['salesChannel', 'orderType']).size().unstack(fill_value=0)

# ---- 4: Engajamento por Tipo de Campanha (badge) ---
status_map = {1: 'Agendado', 2: 'Enviado', 3: 'Recebido', 4: 'Lido', 5: 'Deletado', 6: 'Pendente'}
df_full_campaign_info['status_legivel'] = df_full_campaign_info['status_queue'].map(status_map)
analise_engajamento = df_full_campaign_info.groupby('badge')['status_legivel'].value_counts().unstack(fill_value=0)

# ---- 5: Atraso Médio no Envio de Campanhas ---
analise_atraso = df_campaign_queue['atraso_envio'].dt.total_seconds() / 60
atraso_medio_minutos = analise_atraso.mean()

# ---- 6: ROI por Campanha (Atribuição de Vendas) ---
janela_atribuicao = pd.to_timedelta('3D')
df_vendas_atribuidas = df_integrated[
    (df_integrated['createdAt'] > df_integrated['sendAt']) &
    (df_integrated['createdAt'] <= df_integrated['sendAt'] + janela_atribuicao)
]
analise_roi = df_vendas_atribuidas.groupby('name').agg(
    receita_gerada=('totalAmount', 'sum'),
    pedidos_gerados=('id', 'nunique')
).sort_values(by='receita_gerada', ascending=False)

print("tabelas formatadas.")


Formatando tabelas para analise...
...tabelas formatadas.


In [30]:
# 6. Exibindo cada um dos resultados preparados

print("\n\n--- RESULTADO 1: Pedidos por Hora do Dia ---")
display(analise_pico_horario)

print("\n--- RESULTADO 2: Pedidos por Dia da Semana ---")
display(analise_dia_semana)

print("\n--- RESULTADO 3: Tipo de Pedido x Canal de Venda ---")
display(analise_tipo_canal)

print("\n--- RESULTADO 4: Funil de Engajamento por Tipo de Campanha ---")
display(analise_engajamento)

print(f"\n--- RESULTADO 5: Atraso Médio no Envio das Campanhas: {atraso_medio_minutos:.2f} minutos ---")

print("\n--- RESULTADO 6: ROI por Campanha (Vendas Atribuídas) ---")
display(analise_roi)



--- RESULTADO 1: Pedidos por Hora do Dia ---


,hora_do_dia,qtd_pedidos
0,0,80
1,1,86
2,2,83
3,3,85
4,4,92
5,5,87
6,6,101
7,7,72
8,8,81
9,9,77



--- RESULTADO 2: Pedidos por Dia da Semana ---


,Dia da Semana,Qtd de Pedidos
0,Sunday,311
1,Tuesday,297
2,Monday,295
3,Wednesday,287
4,Saturday,273
5,Friday,272
6,Thursday,265



--- RESULTADO 3: Tipo de Pedido x Canal de Venda ---


orderType,DELIVERY,INDOOR,TAKEOUT
salesChannel,,,
99FOOD,108,104,78
ANOTAAI,97,95,93
DELIVERYVIP,96,84,101
EPADOCA,95,106,95
IFOOD,103,97,86
SITE,98,95,83
WHATSAPP,91,104,91



--- RESULTADO 4: Funil de Engajamento por Tipo de Campanha ---


status_legivel,Agendado,Deletado,Enviado,Lido,Pendente,Recebido
badge,,,,,,
consumption,156,148,159,145,147,137
loyalty,140,133,131,141,152,134
migration,147,126,113,133,149,122
seasonal,126,149,128,134,137,148
winback,150,141,151,126,124,148



--- RESULTADO 5: Atraso Médio no Envio das Campanhas: 91.09 minutos ---

--- RESULTADO 6: ROI por Campanha (Vendas Atribuídas) ---


,receita_gerada,pedidos_gerados
name,,
Campanha Laborum 7K99,199.89,2
Campanha Quisquam VD1Z,144.92,1
Campanha Facere XYLJ,128.24,1
Campanha Tenetur MTEB,124.93,1
Campanha Dolor WG9A,116.55,1
Campanha Debitis RD8S,110.90,1
Campanha Recusandae J0VS,107.77,1
Campanha Ducimus EFRS,106.70,1
Campanha Veritatis 1LSZ,97.36,1
